
# Lab 1 — Data Wrangling & Visualization (Assignment Version)

**Goal:** Practice core **pandas** data wrangling and **Altair** visualization skills without downloading any external datasets.

- ✅ All data is created *inside* this notebook (no internet required).
- 🧩 Tasks mirror the functions and operations used in the reference lab.
- ✍️ Replace the `...` / `raise NotImplementedError()` with your code.
- 🔎 Keep your answers concise and readable.
- 📈 Use **Altair** for plotting tasks.


In [288]:
!pip install numpy
!pip install pandas
!pip install altair

In [289]:

# Setup (do not modify this cell)
import numpy as np
import pandas as pd
import altair as alt

# Make Altair work well in notebooks
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)
print("Libraries imported. Versions -> pandas:", pd.__version__, "| numpy:", np.__version__)


Libraries imported. Versions -> pandas: 2.3.2 | numpy: 2.3.3


## 1) Create in-notebook datasets

In [290]:

# We'll synthesize a small, realistic dataset about city metrics.
# No downloads — everything is generated here.

rng = np.random.default_rng(42)

cities = [
    "Vancouver","Victoria","Kelowna","Abbotsford",
    "Calgary","Edmonton"
]

records = []
for city in cities:
    n = rng.integers(80, 140)  # rows per city
    base_year = 2018
    for i in range(n):
        year = base_year + int(i // 12)  # span multiple years
        month = (i % 12) + 1
        date = pd.Timestamp(year=year, month=month, day=1)
        pop = rng.normal( # synthetic population (in thousands)
            600 if city=="Vancouver" else 100 if city=="Victoria" else 150 if city=="Kelowna" else 180 if city=="Abbotsford" else 1300 if city=="Calgary" else 980,
            20
        )

        # Just practice to write clean codes. We should write like below codes.
        # if city == "Vancouver":
        #     pop = rng.normal(600)
        # else:
        #     pop = rng.normal(600)
        # if city == "Victoria":
        #     pop = rng.normal(150)
        # else:
        #     pop = rng.normal(180)

        median_home_price = rng.normal(
            1200 if city=="Vancouver" else 900 if city=="Victoria" else 700 if city=="Kelowna" else 650 if city=="Abbotsford" else 500 if city=="Calgary" else 420,
            60
        ) * 1000
        unemployment_rate = np.clip(rng.normal(6.0, 1.5), 2.5, 12.0)
        # inject some missingness
        if rng.random() < 0.05:
            unemployment_rate = np.nan
        records.append({
            "city": city,
            "date": date,
            "population_k": max(10, round(pop, 1)),
            "median_home_price": round(median_home_price, 0),
            "unemployment_rate": unemployment_rate
        })

city_df = pd.DataFrame.from_records(records).sort_values(["city","date"]).reset_index(drop=True)

# A small lookup table we can merge later
province_lookup = pd.DataFrame({
    "city": ["Vancouver","Victoria","Kelowna","Abbotsford","Calgary","Edmonton"],
    "province": ["BC","BC","BC","BC","AB","AB"]
})

city_df.head()


,city,date,population_k,median_home_price,unemployment_rate
0,Abbotsford,2018-01-01,203.8,604369.0,5.158869
1,Abbotsford,2018-02-01,148.7,663726.0,7.451199
2,Abbotsford,2018-03-01,148.2,642571.0,7.113043
3,Abbotsford,2018-04-01,158.5,702593.0,6.405143
4,Abbotsford,2018-05-01,217.2,678113.0,6.986832


In [291]:
province_lookup

,city,province
0,Vancouver,BC
1,Victoria,BC
2,Kelowna,BC
3,Abbotsford,BC
4,Calgary,AB
5,Edmonton,AB


In [292]:
city_df.describe()

,date,population_k,median_home_price,unemployment_rate
count,661,661.000000,6.610000e+02,633.000000
mean,2022-09-02 23:01:10.801815296,507.244932,7.175524e+05,5.945491
min,2018-01-01 00:00:00,40.700000,2.736450e+05,2.500000
25%,2020-04-01 00:00:00,141.600000,5.236540e+05,4.982603
50%,2022-08-01 00:00:00,187.100000,6.804240e+05,6.049173
75%,2024-11-01 00:00:00,976.500000,8.796440e+05,6.983427
max,2028-11-01 00:00:00,1356.300000,1.351048e+06,10.370794
std,NaN,461.763845,2.469072e+05,1.527710



## 2) Inspecting data & types

**Tasks** (pandas):
- Show the first and last few rows
- Inspect column data types
- Convert `date` to datetime (if needed)
- Ensure `province` can be added by merging the lookup table


In [293]:

# 2.1 Show the first 5 rows of city_df
first5 = city_df.iloc[:5]
first5


,city,date,population_k,median_home_price,unemployment_rate
0,Abbotsford,2018-01-01,203.8,604369.0,5.158869
1,Abbotsford,2018-02-01,148.7,663726.0,7.451199
2,Abbotsford,2018-03-01,148.2,642571.0,7.113043
3,Abbotsford,2018-04-01,158.5,702593.0,6.405143
4,Abbotsford,2018-05-01,217.2,678113.0,6.986832


In [294]:

# 2.2 Show the last 3 rows of city_df
last3 = city_df[-3:]
last3


,city,date,population_k,median_home_price,unemployment_rate
658,Victoria,2028-04-01,76.8,773001.0,3.207232
659,Victoria,2028-05-01,100.6,892943.0,7.821285
660,Victoria,2028-06-01,107.9,993686.0,4.308329


In [295]:

# 2.3 Print dtypes of city_df
dtypes = city_df.info()
dtypes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   city               661 non-null    object        
 1   date               661 non-null    datetime64[ns]
 2   population_k       661 non-null    float64       
 3   median_home_price  661 non-null    float64       
 4   unemployment_rate  633 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 25.9+ KB


In [296]:

# 2.4 Merge province info into city_df using an inner join on 'city'
# Expected columns include a new 'province' column.
with_province = pd.merge(city_df, province_lookup, on='city', how='inner')
with_province.head()

,city,date,population_k,median_home_price,unemployment_rate,province
0,Abbotsford,2018-01-01,203.8,604369.0,5.158869,BC
1,Abbotsford,2018-02-01,148.7,663726.0,7.451199,BC
2,Abbotsford,2018-03-01,148.2,642571.0,7.113043,BC
3,Abbotsford,2018-04-01,158.5,702593.0,6.405143,BC
4,Abbotsford,2018-05-01,217.2,678113.0,6.986832,BC



## 3) Filtering, sorting, assigning

**Tasks**:
- Filter rows for British Columbia (BC) cities only
- Keep observations from **2020-01-01** onward
- Create a new column `price_per_capita` = median_home_price / (population_k * 1000)
- Sort by `price_per_capita` descending


In [297]:
unique_province = with_province["province"].unique()
unique_province

array(['BC', 'AB'], dtype=object)

In [298]:

# 3.1 Filter BC rows (province == 'BC') and date >= '2020-01-01'
bc_recent = with_province[(with_province["province"]=='BC') & (with_province['date'] >= '2020-01-01')].copy()
bc_recent.head()

,city,date,population_k,median_home_price,unemployment_rate,province
24,Abbotsford,2020-01-01,163.0,783756.0,7.493235,BC
25,Abbotsford,2020-02-01,180.6,636966.0,5.685887,BC
26,Abbotsford,2020-03-01,208.0,660879.0,4.632835,BC
27,Abbotsford,2020-04-01,198.4,689280.0,7.118248,BC
28,Abbotsford,2020-05-01,161.4,597908.0,3.546805,BC


In [299]:

# 3.2 Create 'price_per_capita' and sort descending by it
bc_recent['price_per_capita'] = bc_recent['median_home_price'] / (bc_recent['population_k'] * 1000)
bc_recent = bc_recent.sort_values(by='price_per_capita', ascending=False)
bc_recent.head()


,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita
577,Victoria,2021-07-01,40.7,867390.0,9.630623,BC,21.311794
633,Victoria,2026-03-01,54.4,810202.0,4.615670,BC,14.893419
604,Victoria,2023-10-01,66.3,932282.0,4.448538,BC,14.061569
651,Victoria,2027-09-01,64.3,892639.0,7.493552,BC,13.882411
575,Victoria,2021-05-01,63.4,879864.0,3.013782,BC,13.877981



## 4) Handling missing values

**Tasks**:
- Count missing values in `unemployment_rate` by `city`
- Fill missing `unemployment_rate` with the **group mean** (per city)


In [300]:
bc_recent

,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita
577,Victoria,2021-07-01,40.7,867390.0,9.630623,BC,21.311794
633,Victoria,2026-03-01,54.4,810202.0,4.615670,BC,14.893419
604,Victoria,2023-10-01,66.3,932282.0,4.448538,BC,14.061569
651,Victoria,2027-09-01,64.3,892639.0,7.493552,BC,13.882411
575,Victoria,2021-05-01,63.4,879864.0,3.013782,BC,13.877981
...,...,...,...,...,...,...,...
520,Vancouver,2023-11-01,609.6,1095325.0,7.391158,BC,1.796793
511,Vancouver,2023-02-01,642.6,1122755.0,4.354822,BC,1.747207
525,Vancouver,2024-04-01,634.5,1107968.0,7.295742,BC,1.746206
512,Vancouver,2023-03-01,658.1,1129706.0,5.447627,BC,1.716618


In [301]:

# 4.1 Count missing values of unemployment_rate for each city
missing_counts = bc_recent['unemployment_rate'].isnull().sum()
missing_counts


np.int64(12)

In [302]:
missing_index = bc_recent[bc_recent['unemployment_rate'].isnull()].index.tolist()
missing_index

[566, 603, 630, 636, 647, 354, 499, 517, 492, 504, 489, 507]

In [303]:
missing_rows = bc_recent.loc[missing_index]
missing_rows

,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita
566,Victoria,2020-08-01,68.2,775621.0,NaN,BC,11.372742
603,Victoria,2023-09-01,87.3,932468.0,NaN,BC,10.681191
630,Victoria,2025-12-01,99.7,942100.0,NaN,BC,9.449348
636,Victoria,2026-06-01,105.6,909487.0,NaN,BC,8.612566
647,Victoria,2027-05-01,101.2,821841.0,NaN,BC,8.120958
354,Kelowna,2020-02-01,161.8,630948.0,NaN,BC,3.899555
499,Vancouver,2022-02-01,592.1,1199693.0,NaN,BC,2.026166
517,Vancouver,2023-08-01,601.1,1200005.0,NaN,BC,1.996348
492,Vancouver,2021-07-01,599.2,1160713.0,NaN,BC,1.937104
504,Vancouver,2022-07-01,626.7,1190730.0,NaN,BC,1.900000


In [304]:

# 4.2 Fill missing unemployment_rate with city-wise mean
city_wise_mean = bc_recent.groupby('city')['unemployment_rate'].mean()
print(f'{city_wise_mean}')

filled = bc_recent.copy()
filled['unemployment_rate'] = filled['unemployment_rate'].fillna(filled['city'].map(city_wise_mean))
print(f'{filled[["city","date","unemployment_rate"]].head(10)}')
print(f'{filled.loc[missing_index]}')

city
Abbotsford    5.708858
Kelowna       5.907396
Vancouver     6.064684
Victoria      6.045057
Name: unemployment_rate, dtype: float64
         city       date  unemployment_rate
577  Victoria 2021-07-01           9.630623
633  Victoria 2026-03-01           4.615670
604  Victoria 2023-10-01           4.448538
651  Victoria 2027-09-01           7.493552
575  Victoria 2021-05-01           3.013782
619  Victoria 2025-01-01           5.215922
612  Victoria 2024-06-01           9.079027
605  Victoria 2023-11-01           4.790102
586  Victoria 2022-04-01           6.371499
594  Victoria 2022-12-01           6.393054
          city       date  population_k  median_home_price  unemployment_rate province  price_per_capita
566   Victoria 2020-08-01          68.2           775621.0           6.045057       BC         11.372742
603   Victoria 2023-09-01          87.3           932468.0           6.045057       BC         10.681191
630   Victoria 2025-12-01          99.7           942100.0      


## 5) GroupBy & aggregation

**Tasks**:
- Compute **yearly** mean unemployment rate per city
- Compute **median** of `median_home_price` per `(city, year)`
- Return a tidy DataFrame with columns: `city, year, mean_unemp, med_price`


In [305]:
# 5.1 Derive 'year' from 'date' and perform grouped aggregations
df_yearly = filled.copy()
df_yearly['year'] = df_yearly['date'].dt.year

# yearly = yearly.groupby('city')
yearly = df_yearly.groupby(['city','year'])['unemployment_rate'].mean()

print(f'{yearly.head()}')

print(f'{yearly}')

city        year
Abbotsford  2020    5.459955
            2021    5.034041
            2022    5.653756
            2023    5.838028
            2024    6.062454
Name: unemployment_rate, dtype: float64
city        year
Abbotsford  2020    5.459955
            2021    5.034041
            2022    5.653756
            2023    5.838028
            2024    6.062454
            2025    6.290344
            2026    5.479718
            2027    5.848437
            2028    5.713369
Kelowna     2020    5.983611
            2021    5.758361
            2022    6.513134
            2023    6.525537
            2024    5.721975
            2025    5.310680
            2026    6.098801
            2027    5.292349
            2028    6.564067
Vancouver   2020    5.990203
            2021    6.050251
            2022    5.743125
            2023    6.098064
            2024    6.302753
            2025    7.732968
Victoria    2020    6.691053
            2021    6.312114
            2022    5.61045


## 6) Pivoting and melting

**Tasks**:
- Create a **wide** table with `year` as rows and **cities as columns**,
  values = `mean unemployment rate` (from the yearly result above).
- Then **melt** it back to long form with columns: `year, city, mean_unemp`.


In [306]:
df_yearly

,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita,year
577,Victoria,2021-07-01,40.7,867390.0,9.630623,BC,21.311794,2021
633,Victoria,2026-03-01,54.4,810202.0,4.615670,BC,14.893419,2026
604,Victoria,2023-10-01,66.3,932282.0,4.448538,BC,14.061569,2023
651,Victoria,2027-09-01,64.3,892639.0,7.493552,BC,13.882411,2027
575,Victoria,2021-05-01,63.4,879864.0,3.013782,BC,13.877981,2021
...,...,...,...,...,...,...,...,...
520,Vancouver,2023-11-01,609.6,1095325.0,7.391158,BC,1.796793,2023
511,Vancouver,2023-02-01,642.6,1122755.0,4.354822,BC,1.747207,2023
525,Vancouver,2024-04-01,634.5,1107968.0,7.295742,BC,1.746206,2024
512,Vancouver,2023-03-01,658.1,1129706.0,5.447627,BC,1.716618,2023


In [307]:



# 6.1 Pivot to a wide table (year x city)
# First, we need to convert the yearly Series to a DataFrame for proper pivoting
yearly_df = yearly.reset_index()
wide = yearly_df.pivot(index='year', columns='city', values='unemployment_rate')
wide.head()


city,Abbotsford,Kelowna,Vancouver,Victoria
year,,,,
2020,5.459955,5.983611,5.990203,6.691053
2021,5.034041,5.758361,6.050251,6.312114
2022,5.653756,6.513134,5.743125,5.610454
2023,5.838028,6.525537,6.098064,5.948024
2024,6.062454,5.721975,6.302753,6.449364


In [308]:

# 6.2 Melt back to long form
long_again = wide.reset_index().melt(
    id_vars='year', 
    var_name='city', 
    value_name='unemployment_rate'
)
long_again.head()


,year,city,unemployment_rate
0,2020,Abbotsford,5.459955
1,2021,Abbotsford,5.034041
2,2022,Abbotsford,5.653756
3,2023,Abbotsford,5.838028
4,2024,Abbotsford,6.062454



## 7) Quick queries

**Tasks**:
- Using `.query`, subset `filled` for rows with `unemployment_rate < 5` and `median_home_price > 800000`.
- Show the **top 5 cities** by frequency in that subset (use `value_counts`).


In [309]:
filled

,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita
577,Victoria,2021-07-01,40.7,867390.0,9.630623,BC,21.311794
633,Victoria,2026-03-01,54.4,810202.0,4.615670,BC,14.893419
604,Victoria,2023-10-01,66.3,932282.0,4.448538,BC,14.061569
651,Victoria,2027-09-01,64.3,892639.0,7.493552,BC,13.882411
575,Victoria,2021-05-01,63.4,879864.0,3.013782,BC,13.877981
...,...,...,...,...,...,...,...
520,Vancouver,2023-11-01,609.6,1095325.0,7.391158,BC,1.796793
511,Vancouver,2023-02-01,642.6,1122755.0,4.354822,BC,1.747207
525,Vancouver,2024-04-01,634.5,1107968.0,7.295742,BC,1.746206
512,Vancouver,2023-03-01,658.1,1129706.0,5.447627,BC,1.716618


In [310]:

# 7.1 Query subset
subset = filled.query('unemployment_rate < 5 and median_home_price > 800000')
subset.head()


,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita
633,Victoria,2026-03-01,54.4,810202.0,4.615670,BC,14.893419
604,Victoria,2023-10-01,66.3,932282.0,4.448538,BC,14.061569
575,Victoria,2021-05-01,63.4,879864.0,3.013782,BC,13.877981
605,Victoria,2023-11-01,71.5,926779.0,4.790102,BC,12.961944
572,Victoria,2021-02-01,84.4,925818.0,4.722694,BC,10.969408


In [311]:

# 7.2 Top 5 cities by frequency in subset
top5 = subset.value_counts('city')
top5


city
Victoria      25
Vancouver     15
Abbotsford     1
Kelowna        1
Name: count, dtype: int64


## 8) Visualization with Altair

**Tasks**:
- **Line chart**: For one city of your choice, plot `median_home_price` over `date`.
- **Bar chart**: For 2022 only, plot **mean unemployment rate** by city.
- **Scatter plot**: For 2021–2023, show `population_k` vs. `median_home_price`, color by `city`, add tooltips.


In [312]:
df_yearly

,city,date,population_k,median_home_price,unemployment_rate,province,price_per_capita,year
577,Victoria,2021-07-01,40.7,867390.0,9.630623,BC,21.311794,2021
633,Victoria,2026-03-01,54.4,810202.0,4.615670,BC,14.893419,2026
604,Victoria,2023-10-01,66.3,932282.0,4.448538,BC,14.061569,2023
651,Victoria,2027-09-01,64.3,892639.0,7.493552,BC,13.882411,2027
575,Victoria,2021-05-01,63.4,879864.0,3.013782,BC,13.877981,2021
...,...,...,...,...,...,...,...,...
520,Vancouver,2023-11-01,609.6,1095325.0,7.391158,BC,1.796793,2023
511,Vancouver,2023-02-01,642.6,1122755.0,4.354822,BC,1.747207,2023
525,Vancouver,2024-04-01,634.5,1107968.0,7.295742,BC,1.746206,2024
512,Vancouver,2023-03-01,658.1,1129706.0,5.447627,BC,1.716618,2023


In [313]:

# 8.1 Line chart of median_home_price over time for a chosen city
chosen_city = "Vancouver"  # you may change
data_city = filled[filled['city']==chosen_city]  # TODO
line_chart = alt.Chart(data_city).mark_line().encode(
    x="date:T",
    y="median_home_price:Q",
    tooltip=["date","median_home_price"]
).properties(width=600, height=300)
line_chart


alt.Chart(...)

In [314]:

# 8.2 Bar chart: mean unemployment rate by city for 2022
year_2022 = df_yearly[df_yearly['year']==2022]  # TODO
bar_chart = alt.Chart(year_2022).mark_bar().encode(
    x="city:N",
    y="mean(unemployment_rate):Q",
    tooltip=["city","mean(unemployment_rate)"]
).properties(width=600, height=300)
bar_chart


alt.Chart(...)

In [315]:

# 8.3 Scatter: population_k vs median_home_price for years 2021–2023, colored by city
mask = (filled['date'].dt.year >= 2021) & (filled['date'].dt.year <= 2023)
scat = alt.Chart(filled[mask]).mark_circle().encode(
    x="population_k:Q",
    y="median_home_price:Q",
    color="city:N",
    tooltip=["city","date","population_k","median_home_price"]
).properties(width=600, height=300)
scat


alt.Chart(...)


## 9) Mini `read_csv` task (no external files)

To practice `pd.read_csv` without downloading data, read from a **string** using `io.StringIO`.

**Task:** Parse the mini CSV into a DataFrame and compute the average score by `city`.


In [316]:

import io

csv_text = """city,score
Vancouver,88
Calgary,77
Edmonton,82
Victoria,91
Kelowna,85
Abbotsford,79
"""

# 9.1 Read the CSV string into a DataFrame named mini
mini = pd.read_csv(io.StringIO(csv_text))

mini.head()


,city,score
0,Vancouver,88
1,Calgary,77
2,Edmonton,82
3,Victoria,91
4,Kelowna,85


In [317]:
mini

,city,score
0,Vancouver,88
1,Calgary,77
2,Edmonton,82
3,Victoria,91
4,Kelowna,85
5,Abbotsford,79


In [318]:

# 9.2 Compute the average score by city (it's one row per city here, but pretend!)
avg_scores = mini.groupby('city')['score'].mean()
avg_scores


city
Abbotsford    79.0
Calgary       77.0
Edmonton      82.0
Kelowna       85.0
Vancouver     88.0
Victoria      91.0
Name: score, dtype: float64

## 10) Lightweight self-checks

In [319]:

# A few basic checks to help you verify your work. Feel free to add more.
assert "province" in with_province.columns, "Did you merge the province column?"

assert (with_province["date"].dtype == "datetime64[ns]") or np.issubdtype(with_province["date"].dtype, np.datetime64), "Ensure 'date' is datetime"

assert "price_per_capita" in bc_recent.columns, "Did you create price_per_capita?"

assert set(wide.columns) - {"year"}, "The pivot should create city columns"
print("\nBasic checks passed (or adjust after solving tasks).")



Basic checks passed (or adjust after solving tasks).



---

### Submission notes
- Run **Kernel → Restart & Run All** before exporting.
- Export your notebook as **HTML** and **.ipynb**.
- Make sure all cells execute without errors.

**Good luck!** 🎉
